<a href="https://colab.research.google.com/github/staaason/Labs_crypto/blob/main/lab2/Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Завдання 1**

Реалiзуйте криптосистеми AES-128, AES-192 i AES-256, включаючи раундовi
функцiї та їх оберненi, алгоритми розподiлу ключiв, шифрування та дешифрування.

In [5]:
import math
import random
import os


class AES:
    def __init__(self, input_key, block_length):
        '''
          Nb: number of columns (32-bit words) comprising the State. For this
          standard, Nb = 4.
          Nk: number of 32-bit words comprising the Cipher Key. For this
          standard, Nk = 4, 6, or 8.
          Nr: number of rounds, which is a function of Nk and Nb (which is
          fixed). For this standard, Nr = 10, 12, or 14.
        '''
        parameters = {128:[4, 4, 10], 192:[6, 4, 12], 256:[8, 4, 14]}
        self.Nk, self.Nb, self.Nr = parameters[block_length]
        self.key = input_key
        self.key_matrix = self.fill_key(input_key)
        self.key_schedule = []
        self.sBox = [
        0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
        0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
        0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
        0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
        0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
        0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
        0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
        0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
        0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
        0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
        0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
        0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
        0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
        0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
        0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
        0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16
        ]
        self.inv_sBox = [
        0x52, 0x09, 0x6a, 0xd5, 0x30, 0x36, 0xa5, 0x38, 0xbf, 0x40, 0xa3, 0x9e, 0x81, 0xf3, 0xd7, 0xfb,
        0x7c, 0xe3, 0x39, 0x82, 0x9b, 0x2f, 0xff, 0x87, 0x34, 0x8e, 0x43, 0x44, 0xc4, 0xde, 0xe9, 0xcb,
        0x54, 0x7b, 0x94, 0x32, 0xa6, 0xc2, 0x23, 0x3d, 0xee, 0x4c, 0x95, 0x0b, 0x42, 0xfa, 0xc3, 0x4e,
        0x08, 0x2e, 0xa1, 0x66, 0x28, 0xd9, 0x24, 0xb2, 0x76, 0x5b, 0xa2, 0x49, 0x6d, 0x8b, 0xd1, 0x25,
        0x72, 0xf8, 0xf6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xd4, 0xa4, 0x5c, 0xcc, 0x5d, 0x65, 0xb6, 0x92,
        0x6c, 0x70, 0x48, 0x50, 0xfd, 0xed, 0xb9, 0xda, 0x5e, 0x15, 0x46, 0x57, 0xa7, 0x8d, 0x9d, 0x84,
        0x90, 0xd8, 0xab, 0x00, 0x8c, 0xbc, 0xd3, 0x0a, 0xf7, 0xe4, 0x58, 0x05, 0xb8, 0xb3, 0x45, 0x06,
        0xd0, 0x2c, 0x1e, 0x8f, 0xca, 0x3f, 0x0f, 0x02, 0xc1, 0xaf, 0xbd, 0x03, 0x01, 0x13, 0x8a, 0x6b,
        0x3a, 0x91, 0x11, 0x41, 0x4f, 0x67, 0xdc, 0xea, 0x97, 0xf2, 0xcf, 0xce, 0xf0, 0xb4, 0xe6, 0x73,
        0x96, 0xac, 0x74, 0x22, 0xe7, 0xad, 0x35, 0x85, 0xe2, 0xf9, 0x37, 0xe8, 0x1c, 0x75, 0xdf, 0x6e,
        0x47, 0xf1, 0x1a, 0x71, 0x1d, 0x29, 0xc5, 0x89, 0x6f, 0xb7, 0x62, 0x0e, 0xaa, 0x18, 0xbe, 0x1b,
        0xfc, 0x56, 0x3e, 0x4b, 0xc6, 0xd2, 0x79, 0x20, 0x9a, 0xdb, 0xc0, 0xfe, 0x78, 0xcd, 0x5a, 0xf4,
        0x1f, 0xdd, 0xa8, 0x33, 0x88, 0x07, 0xc7, 0x31, 0xb1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xec, 0x5f,
        0x60, 0x51, 0x7f, 0xa9, 0x19, 0xb5, 0x4a, 0x0d, 0x2d, 0xe5, 0x7a, 0x9f, 0x93, 0xc9, 0x9c, 0xef,
        0xa0, 0xe0, 0x3b, 0x4d, 0xae, 0x2a, 0xf5, 0xb0, 0xc8, 0xeb, 0xbb, 0x3c, 0x83, 0x53, 0x99, 0x61,
        0x17, 0x2b, 0x04, 0x7e, 0xba, 0x77, 0xd6, 0x26, 0xe1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0c, 0x7d
        ]

    def fill_key(self, array):
        return [[array[j+i*self.Nk] for j in range(self.Nk)] for i in range(4)]

    def fill_state(self, input_array):
        return [[input_array[i+j*self.Nb] for j in range(self.Nb)] for i in range(4)]

    def transpose_key(self, arr):
        return [[arr[j][i] for j in range(len(arr))] for i in range(len(arr[0]))]
    

    '''
    Transformation in the Cipher and Inverse Cipher in which a Round
    Key is added to the State using an XOR operation. The length of a
    Round Key equals the size of the State (i.e., for Nb = 4, the Round
    Key length equals 128 bits/16 bytes).
    '''
    def add_round_key(self, arr, rk):
        rk = self.transpose_key(rk)
        temp = [0]*self.Nb
        for i in range(self.Nb):
            temp[i] = self.bins_xor(arr[i], rk[i])
        return  temp

    def sub_bytes(self, array):
        return [[self.sBox[i] for i in row] for row in array]

    def shift(self, array, pos):
        for i in range(pos):
            array.append(array.pop(0))
        return array

    def shift_rows(self, arr):
        temp = [[0]*4]*self.Nb
        for i in range(self.Nb):
            temp[i] = self.shift(arr[i], i)
        return temp

    def inv_shift(self, array, pos):
        for i in range(len(array) - pos):
            array.append(array.pop(0))
        return array


    '''
    Transformation in the Inverse Cipher that is the inverse of ShiftRows().
    '''
    def inv_shift_rows(self, arr):
        for i in range(self.Nb):
            self.inv_shift(arr[i], i)
        return arr
    '''
      Function is used to multiply a number by 2 in the Galois field. 
    '''
    def xtime(self, x):
        return (x << 1) ^ (((x >> 7) & 0x01) * 0x1b) 

    def multiply(self, x, y):
        result = (y & 0x01) * x
        result ^= (y >> 1 & 0x01) * self.xtime(x)
        result ^= (y >> 2 & 0x01) * self.xtime(self.xtime(x))
        result ^= (y >> 3 & 0x01) * self.xtime(self.xtime(self.xtime(x)))
        result ^= (y >> 4 & 0x01) * self.xtime(self.xtime(self.xtime(self.xtime(x))))
        return result & 0xff

    def mix_column(self, col):
        res = []
        res.append(self.multiply(col[0], 0x02) ^ self.multiply(col[1], 0x03) ^ self.multiply(col[2], 0x01) ^ self.multiply(col[3], 0x01))
        res.append(self.multiply(col[0], 0x01) ^ self.multiply(col[1], 0x02) ^ self.multiply(col[2], 0x03) ^ self.multiply(col[3], 0x01))
        res.append(self.multiply(col[0], 0x01) ^ self.multiply(col[1], 0x01) ^ self.multiply(col[2], 0x02) ^ self.multiply(col[3], 0x03))
        res.append(self.multiply(col[0], 0x03) ^ self.multiply(col[1], 0x01) ^ self.multiply(col[2], 0x01) ^ self.multiply(col[3], 0x02))
        return res
  
    def inv_mix_column(self, col):
        res = []
        res.append(self.multiply(col[0], 0x0e) ^ self.multiply(col[1], 0x0b) ^ self.multiply(col[2], 0x0d) ^ self.multiply(col[3], 0x09))
        res.append(self.multiply(col[0], 0x09) ^ self.multiply(col[1], 0x0e) ^ self.multiply(col[2], 0x0b) ^ self.multiply(col[3], 0x0d))
        res.append(self.multiply(col[0], 0x0d) ^ self.multiply(col[1], 0x09) ^ self.multiply(col[2], 0x0e) ^ self.multiply(col[3], 0x0b))
        res.append(self.multiply(col[0], 0x0b) ^ self.multiply(col[1], 0x0d) ^ self.multiply(col[2], 0x09) ^ self.multiply(col[3], 0x0e))
        return res

    '''
    Transformation in the Cipher that takes all of the columns of the
    State and mixes their data (independently of one another) to
    produce new columns.  
    '''
    def mix_columns(self, arr):
        temp = arr.copy()
        temp = [[temp[j][i] for j in range(len(temp))] for i in range(len(temp[0]))]
        result = arr.copy()
        for i in range(len(arr)):
            result[i] = self.mix_column(temp[i])
        result = [[result[j][i] for j in range(len(result))] for i in range(len(result[0]))]
        return result
    '''
    Transformation in the Inverse Cipher that is the inverse of mix_columns().
    '''
    def inv_mix_columns(self, arr):
        temp = arr.copy()
        temp = [[temp[j][i] for j in range(len(temp))] for i in range(len(temp[0]))]
        result = arr.copy()
        for i in range(len(arr)):
            result[i] = self.inv_mix_column(temp[i])
        result = [[result[j][i] for j in range(len(result))] for i in range(len(result[0]))]
        return result

    '''
    Transformation in the Inverse Cipher that is the inverse of SubBytes().
    '''
    def inv_sub_bytes(self, array):
        return [[self.inv_sBox[i] for i in row] for row in array]

    def sub_word(self, word):
        return [self.sBox[i] for i in word]

    def rot_word(self, word):
        word.append(word.pop(0))
        return word

    def bins_xor(self, word1, word2):
        arr = [word1[i]^word2[i] for i in range(4)]
        return arr
    '''
    Function is used to perform a bitwise XOR operation between a 32-bit word and 
    the Rcon value of a specific round in the key expansion algorithm.
    '''
    def xor_rcon(self, temp, r0):
        rc = r0
        return [temp[0]^rc] +temp[1:]

    '''
     Function  is responsible for creating a list of round keys, which are derived from the input key.
    '''
    def create_round_keys(self):
        Rcon = [0x00, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40,
        0x80, 0x1B, 0x36, 0x6C, 0xD8, 0xAB, 0x4D, 0x9A,
        0x2F, 0x5E, 0xBC, 0x63, 0xC6, 0x97, 0x35, 0x6A,
        0xD4, 0xB3, 0x7D, 0xFA, 0xEF, 0xC5, 0x91, 0x39]
        w = [[0]*4]*(self.Nb*(self.Nr+1))
        i = 0
        while (i < self.Nk):
            w[i] = [self.key[4*i], self.key[4*i+1],self.key[4*i+2], self.key[4*i+3]]
            i = i + 1
        i = self.Nk
        while (i < self.Nb * (self.Nr + 1)):
            temp = w[i - 1].copy()
            if (i % self.Nk == 0):
                temp = self.rot_word(temp)
                temp = self.sub_word(temp)
                temp = self.xor_rcon(temp, Rcon[math.floor(i / self.Nk)])
            elif (self.Nk > 6 and i%self.Nk == 4):
                temp = self.sub_word(temp)
            w[i] = self.bins_xor(w[i - self.Nk], temp)
            i = i + 1
        return w

    def encrypt(self, input):
        state = self.fill_state(input)
        self.key_schedule = self.create_round_keys()
        temp = state.copy()
        temp = self.add_round_key(temp, self.key_schedule[:self.Nb])     
        for r in range(1, self.Nr):
          temp = self.sub_bytes(temp)
          temp = self.shift_rows(temp)
          temp = self.mix_columns(temp)   
          temp = self.add_round_key(temp, self.key_schedule[r*self.Nb:(r+1)*self.Nb])

        temp = self.sub_bytes(temp)
        temp = self.shift_rows(temp)
        temp = self.add_round_key(temp, self.key_schedule[self.Nr*self.Nb:(self.Nr+1)*self.Nb])
        return temp

    def decrypt(self, ct):
        temp = ct
        temp = self.add_round_key(temp, self.key_schedule[self.Nr*self.Nb:(self.Nr+1)*self.Nb])
        for r in range(self.Nr-1, 0, -1):
            temp = self.inv_shift_rows(temp)
            temp = self.inv_sub_bytes(temp)
            temp = self.add_round_key(temp, self.key_schedule[r*self.Nb:(r + 1)*(self.Nb)])
            temp = self.inv_mix_columns(temp)
        temp = self.inv_sub_bytes(temp)
        temp = self.inv_shift_rows(temp)
        temp = self.add_round_key(temp, self.key_schedule[:self.Nb])
        return temp

    def show(self, arr):
        array = [[hex(arr[j][i]) for j in range(len(arr))] for i in range(len(arr[0]))]
        hex_list = [item for sublist in array for item in sublist]
        hex_str = ''.join(hex_list).replace('0x', '')
        print(hex_str)


def convert_hex_to_bytes_arr(hex_str):
  bytes_obj = bytes.fromhex(hex_str)
  return [int(byte) for byte in bytes_obj]

plaintext = "00112233445566778899aabbccddeeff"
key = "000102030405060708090a0b0c0d0e0f"
state = convert_hex_to_bytes_arr(plaintext)
key = convert_hex_to_bytes_arr(key)

aes128 = AES(key, 128)
ct128 = aes128.encrypt(state)
print("AES-128 encrypted:")
aes128.show(ct128)
pt128 = aes128.decrypt(ct128)
print("AES-128 decrypted:")
aes128.show(pt128)

key2 = "000102030405060708090a0b0c0d0e0f1011121314151617"
key2 = convert_hex_to_bytes_arr(key2)
aes192 = AES(key2, 192)
ct192 = aes192.encrypt(state)
print("AES-192 encrypted:")
aes192.show(ct192)
pt192 = aes192.decrypt(ct192)
print("AES-192 decrypted:")
aes192.show(pt192)

key3 = "000102030405060708090a0b0c0d0e0f101112131415161718191a1b1c1d1e1f"
key3 = convert_hex_to_bytes_arr(key3)
aes256 = AES(key3, 256)
ct256 = aes256.encrypt(state)
print("AES-256 encrypted:")
aes256.show(ct256)
pt256 = aes256.decrypt(ct256)
print("AES-256 decrypted:")
aes256.show(pt256)







AES-128 encrypted:
69c4e0d86a7b430d8cdb78070b4c55a
AES-128 decrypted:
0112233445566778899aabbccddeeff
AES-192 encrypted:
dda97ca4864cdfe06eaf70a0ecd7191
AES-192 decrypted:
0112233445566778899aabbccddeeff
AES-256 encrypted:
8ea2b7ca516745bfeafc49904b496089
AES-256 decrypted:
0112233445566778899aabbccddeeff


**Завдання 2**

Для кожної з криптосистем AES-128, AES-192 i AES-256 знайдiть, скiльки в
середньому (серед 1000 випадкових випробувань) бiт криптотексту змiнюється
при змiнi
а) одного бiта вiдкритого тексту;
б) одного бiта ключа.

In [3]:
def count_bits_changed(outputs1, outputs2):
    total_bits_changed = 0
    
    for row1, row2 in zip(outputs1, outputs2):
        for element1, element2 in zip(row1, row2):
            binary1 = bin(element1)[2:].zfill(8)
            binary2 = bin(element2)[2:].zfill(8)
            
            for bit1, bit2 in zip(binary1, binary2):
                if bit1 != bit2:
                    total_bits_changed += 1
                    
    return total_bits_changed


def change_bit_state(key_size):
    key_ss = key_size // 8  
    key = os.urandom(key_ss)
    key_list = [int.from_bytes(key[i:i+1], byteorder='big') for i in range(len(key))]
    num_trials = 1000
    num_changed_bits = 0
    aes = AES(key_list, key_size)
    for i in range(num_trials):
        plaintext = [random.randint(0, 255) for _ in range(16)]
        ciphertext1 = aes.encrypt(plaintext)
        plaintext[random.randint(0, 15)] = random.randint(0, 255)
        ciphertext2 = aes.encrypt(plaintext)
        num_changed_bits += count_bits_changed(ciphertext1, ciphertext2)

    return num_changed_bits / num_trials


def change_key(key):
    rn = random.randint(0, len(key) - 1)
    key[rn] = random.randint(0, 255)
    return key

def change_bit_key(key_size):
    num_trials = 1000
    num_changed_bits = 0
    key_ss = key_size // 8  
    for i in range(num_trials):
        key = os.urandom(key_ss)
        key_list = [int.from_bytes(key[i:i+1], byteorder='big') for i in range(len(key))]
        plaintext = [random.randint(0, 255) for _ in range(16)]
        ciphertext1 = AES( key_list, key_size).encrypt(plaintext)
        key2 = change_key(key_list)
        ciphertext2 = AES(key2, key_size).encrypt(plaintext)
        num_changed_bits += count_bits_changed(ciphertext1, ciphertext2)
    return num_changed_bits/num_trials

def get_means():
    plaintext_sizes = [128, 192, 256] # length block is 128 bit
    key_sizes = [128, 192, 256]
    for p_size in plaintext_sizes:
        print(f"mean for {p_size}, plaintext changed:", change_bit_state(p_size))
    for k_size in key_sizes:
        print(f"mean for {k_size}, key changed:", change_bit_key(k_size))


get_means()

mean for 128, plaintext changed: 63.79
mean for 192, plaintext changed: 63.945
mean for 256, plaintext changed: 64.314
mean for 128, key changed: 63.787
mean for 192, key changed: 63.551
mean for 256, key changed: 63.674
